# Data Cleaning for IMDB Dataset

Author: Jennifer Le  
Date: 1/22/25

In [1]:
import pandas as pd
from pandasql import sqldf
import numpy 

'''
The code below was used to read in the original titles dataset.
To save space, this 11 million row, nearly 1GB dataset is replaced by a
smaller filtered dataset. 

# Read in original dataset and delete irrelevant columns and rows
df = pd.read_csv("title.basics.tsv", delimiter='\t', low_memory=False)
df = df.drop(columns=['isAdult','originalTitle'])

# Remove rows where titleType is in list, the ~ symbol negates the isin
print(df['titleType'].unique())
df = df[
    ~df["titleType"].isin(
        ['tvEpisode','video','videoGame','tvPilot','tvSpecial']
    )
]
df.to_csv("titles_reduced.csv",index=False)
'''

'\nThe code below was used to read in the original titles dataset.\nTo save space, this 11 million row, nearly 1GB dataset is replaced by a\nsmaller filtered dataset. \n\n# Read in original dataset and delete irrelevant columns and rows\ndf = pd.read_csv("title.basics.tsv", delimiter=\'\t\', low_memory=False)\ndf = df.drop(columns=[\'isAdult\',\'originalTitle\'])\n\n# Remove rows where titleType is in list, the ~ symbol negates the isin\nprint(df[\'titleType\'].unique())\ndf = df[\n    ~df["titleType"].isin(\n        [\'tvEpisode\',\'video\',\'videoGame\',\'tvPilot\',\'tvSpecial\']\n    )\n]\ndf.to_csv("titles_reduced.csv",index=False)\n'

In [2]:
# add files from google drive to correct folder to 
# run code block without file-not-found error

df_titles = pd.read_csv('../../titles_reduced.csv')
df_ratings = pd.read_csv('../../ratings.tsv', delimiter='\t', low_memory=False)
df_crew = pd.read_csv('../../crew.tsv', delimiter='\t', low_memory=False)

print(df_titles.shape)
print(df_ratings.shape)
print(df_crew.shape)

df_combined = pd.merge(df_titles, df_ratings, on='tconst', how='inner')
df_combined = pd.merge(df_combined, df_crew, on='tconst', how='inner')

# replace null values with 0 or empty space
df_combined['startYear'] = df_combined['startYear'].replace({'\\N': '0'}) 
df_combined['endYear'] = df_combined['endYear'].replace({'\\N': '0'})        
df_combined['runtimeMinutes'] = df_combined['runtimeMinutes'].replace({'\\N':'0'})
df_combined['genres'] = df_combined['genres'].replace({'\\N':''}) 
df_combined['directors'] = df_combined['directors'].replace({'\\N':''}) 
df_combined['writers'] = df_combined['writers'].replace({'\\N':''}) 

# convert types to save space
df_combined['endYear'] = df_combined['endYear'].astype('int32')   
df_combined['runtimeMinutes'] = df_combined['runtimeMinutes'].astype('int32')       
df_combined['startYear'] = df_combined['startYear'].astype('int32')  

df_combined.to_csv('result.csv', index=False)
print(df_combined.shape)
print(df_combined.dtypes)

df_combined.to_csv('imdb_dataset_original.csv', index=False)

(2238148, 7)
(1526047, 3)
(11401626, 3)


In [22]:
df_combined = pd.read_csv('imdb_dataset_original.csv')
df_names = pd.read_csv('../../names.csv', low_memory=False)

In [23]:
# extract columns to make fact tables
df_genres = df_combined[['tconst','genres']]
df_directors = df_combined[['tconst','directors']]
df_writers = df_combined[['tconst','writers']]

# drop columns from dimension table because 
# it will be repetive once fact tables are set up
df_combined = df_combined.drop(columns=['genres','writers','directors'])
df_combined.to_csv('imdb_dataset_transformed.csv',index=False)

# split columns 
df_genres_split = df_genres['genres'].str.split(pat=',',expand=True, n=2)
df_directors_split = df_directors['directors'].str.split(pat=',',expand=True, n=3)
df_writers_split = df_writers['writers'].str.split(pat=',',expand=True, n=3)

# add new columns for the splited genres
df_genres['genres'] = df_genres_split[0]
df_genres['genre2'] = df_genres_split[1]
df_genres['genre3'] = df_genres_split[2]

# add new columns for the splited directors
df_directors['directors'] = df_directors_split[0]
df_directors['director2'] = df_directors_split[1]
df_directors['director3'] = df_directors_split[2]

# add new columns for the splited writers
df_writers['writers'] = df_writers_split[0]
df_writers['writer2'] = df_writers_split[1]
df_writers['writer3'] = df_writers_split[2]

# change column names
df_genres = df_genres.rename(columns={'genres':'genre1'})
df_directors = df_directors.rename(columns={'directors':'director1'})
df_writers = df_writers.rename(columns={'writers':'writer1'})

# function to replace name id with actual name
def replace_id_with_names(df, df_names, column_name):
    df = pd.merge(df,df_names, 
        left_on=column_name, right_on='nconst',how ='left')

    df[column_name] = df['primaryName']
    df = df.drop(columns=['nconst','primaryName'])

    return df

# replace id of directors and writers with names
df_directors = replace_id_with_names(df_directors, df_names, 'director1')
df_directors = replace_id_with_names(df_directors, df_names, 'director2')
df_directors = replace_id_with_names(df_directors, df_names, 'director3')
df_writers = replace_id_with_names(df_writers, df_names, 'writer1')
df_writers = replace_id_with_names(df_writers, df_names, 'writer2')
df_writers = replace_id_with_names(df_writers, df_names, 'writer3')

def create_fact_tables(df, new_name, csv_file_name):
    # transform table to long format so all category values are in one column
    df = pd.melt(df, id_vars='tconst', value_vars=df.columns[1:])
    df = df.rename(columns={'variable':'col_num','value':new_name})

    # delete rows with no genre, director, or writer
    df = df[df[new_name].notna()]

    # write transformed table to csv
    df.to_csv(csv_file_name,index=False)

create_fact_tables(df_genres, 'genre', 'genres.csv')
create_fact_tables(df_directors, 'director', 'directors.csv')
create_fact_tables(df_writers, 'writer', 'writers.csv')